In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import f1_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]

In [3]:
dset_idx = 0

In [4]:
seed = 123
rand = np.random.RandomState()

In [5]:
condition_map = {"healthy": 0, "tumor": 1}

# Load normalized matrisome count data

In [6]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
coldata_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/coldata.tsv", sep="\t")
counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df, "geneID", "sample_name"
)
print(counts_t_df.shape)

(272, 1009)


In [7]:
joined_df = (
    pd.merge(coldata_df, counts_t_df, on="sample_name")
        .set_index("sample_name")
        .drop("data_source", axis=1)
        .assign(condition = lambda df: df.condition.apply(lambda x: condition_map[x]))
)

In [8]:
joined_df.condition.value_counts()

1    259
0     13
Name: condition, dtype: int64

# Prep data

In [9]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [10]:
rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.condition.value_counts()).reset_index()
        .rename(columns={"index": "label", "condition": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="macro")

mc_baseline = opt.mc_classification_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="macro"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.487758945386064
Monte Carlo baseline: 0.4982187021790945


# SMBO

In [11]:
def objective(h_params, X, y, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = RandomForestClassifier(
        n_estimators=h_params[0],
        max_depth=h_params[1],
        max_features=h_params[2],
        min_samples_split=h_params[3],
        min_samples_leaf=h_params[4],
        bootstrap=h_params[5],
        n_jobs=-1,
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [12]:
space = [
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(int(10), int(100), name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(4), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf"),
    Categorical([True, False], name="bootstrap")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [13]:
scoring_default = "f1_macro"
callback_file = f"{unified_dsets[dset_idx]}_opt_rfc_cancer_y_n_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[742, 49, 'log2', 3, 2, True]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.3350
Function value obtained: -0.8721
Current minimum: -0.8721
Iteration No: 2 started. Evaluating function at random point.
[470, 62, 'auto', 3, 2, False]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.0174
Function value obtained: -0.9879
Current minimum: -0.9879
Iteration No: 3 started. Evaluating function at random point.
[320, 73, 'log2', 3, 2, False]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.3635
Function value obtained: -0.9879
Current minimum: -0.9879
Iteration No: 4 started. Evaluating function at random point.
[741, 49, 'log2', 3, 2, False]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.6352
Function value obtained: -0.9879
Current minimum: -0.9879
Iteration No: 5 started. Evaluating function at random point.
[475, 69, 'log2', 2, 3, True]
Itera

KeyboardInterrupt: 